In [ ]:
from datetime import datetime
from IPython.display import display, HTML
import re
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, fixed, interact, interactive, interact_manual, Label, Layout, Textarea

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 25ex !important; font-weight: bold;}
.widget-checkbox {padding-right: 300px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))


class ConfigForm(object):
    # Define widgets
    caption = widgets.HTML(value='<h3>Configure Your Action</h3>')
    button = widgets.Button(description='Submit')
    action = widgets.Dropdown(options=['Insert', 'Add to Existing Collection'], value='Insert')
    datatype = widgets.RadioButtons(options=['RawData', 'ProcessedData'], value='ProcessedData')
    collection = widgets.Text(value='')
    relationships = widgets.Textarea(value='', placeholder="List each relationship on a separate line.")
    rights = widgets.Text(value='')
    OCR = widgets.Checkbox(value=False)
    source_files = widgets.Textarea(value='', placeholder="List each manifest file on a separate line.")
    processes = widgets.Textarea(value='', placeholder="List each process on a separate line.")
    title = widgets.Text(value='')
    altTitle = widgets.Text(value='')
    description = widgets.Textarea(value='')
    date = widgets.Textarea(placeholder='List each date on a separate line.\nDate ranges should use the separate start and end dates with commas.\nValid formats are YYYY-MM-DD and YYYY-MM-DDTHH:MM:SS.')
    label = widgets.Text(value='')
    precise = widgets.Checkbox(value=False)
    notes = widgets.Textarea(placeholder='List each note on a separate line.')
    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    form_items = [
        Box([Label(value='Action:'), action], layout=flex),
        Box([Label(value='Node Type:'), datatype], layout=flex),
        Box([Label(value='collection:'), collection], layout=flex),
        Box([Label(value='relationships:'), relationships], layout=flex),
        Box([Label(value='rights:'), rights], layout=flex),
        Box([Label(value='Text from OCR:'), OCR], layout=flex),
        Box([Label(value='Manifest Files:'), source_files], layout=flex),
        Box([Label(value='Processes:'), processes], layout=flex),
        Box([Label(value='title (optional):'), title], layout=flex),
        Box([Label(value='altTitle (optional):'), altTitle], layout=flex),
        Box([Label(value='description (optional):'), description], layout=flex),
        Box([Label(value='date (optional):'), date], layout=flex),
        Box([Label(value='label (optional):'), label], layout=flex),
        Box([Label(value='notes (optional):'), notes], layout=flex)
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self.action = ConfigForm.form_items[0]
        self.datatype = ConfigForm.form_items[1]
        self.collection = ConfigForm.form_items[2]
        self.relationships = ConfigForm.form_items[3]
        self.rights = ConfigForm.form_items[4]
        self.ocr = ConfigForm.form_items[5]
        self.source_files = ConfigForm.form_items[6]
        self.processes = ConfigForm.form_items[7]
        self.title = ConfigForm.form_items[8]
        self.altTitle = ConfigForm.form_items[9]
        self.description = ConfigForm.form_items[10]
        self.date = ConfigForm.form_items[11]
        self.label = ConfigForm.form_items[12]
        self.notes = ConfigForm.form_items[13]

        # Modify widget visibility when the form is changed
        def change_visibility(change):
            box = self.form
            new = str(ConfigForm.action.value) + str(ConfigForm.datatype.value)
            if new == 'InsertRawData':
                for item in box.children[1:6]:
                    item.add_class('visible').remove_class('hidden')
                box.children[7].remove_class('visible').add_class('hidden')
            elif new == 'Add to Existing CollectionRawData':
                for item in box.children[1:6]:
                    item.add_class('visible').remove_class('hidden')
                box.children[7].remove_class('visible').add_class('hidden')
            elif new == 'InsertProcessedData':
                for item in box.children[1:3]:
                    item.add_class('visible').remove_class('hidden')
                for item in box.children[3:6]:
                    item.remove_class('visible').add_class('hidden')
                box.children[7].add_class('visible').remove_class('hidden')
            else:
                for item in box.children[1:3]:
                    item.add_class('visible').remove_class('hidden')
                for item in box.children[3:6]:
                    item.remove_class('visible').add_class('hidden')
                box.children[7].add_class('visible').remove_class('hidden')
                
        def split_list(str):
            seq = str.split('\n')
            for key, value in enumerate(seq):
                seq[key] = value.strip()
            return seq

        # Ensure that a date is correctly formatted and start dates precede end dates
        def check_date_format(date):
            items = date.replace(' ', '').split(',')
            for item in items:
                if len(item) > 10:
                    format = '%Y-%m-%dT%H:%M:%S'
                else:
                    format = '%Y-%m-%d'
                try:
                    if item != datetime.strptime(item, format).strftime(format):
                        raise ValueError
                    return True
                except ValueError:
                    print('The date value ' + item + ' is in an incorrect format. Use YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS.')
            if len(items) == 2:
                try:
                    assert items[1] > items[0]
                except:
                    print('Your end date ' + items[1] + ' must be after your start date ' + items[0] + '.')


        # Transform textarea with dates to a valid schema array
        def process_dates(datefield):
            date = datefield.strip().split('\n')
            new_date = []
            d = {}

            # Validate all the date formats individually
            for item in date:
                check_date_format(item)

            # Determine if the datelist is a mix of normal and precise dates
            contains_precise = []
            for item in date:
                if ',' in item:
                    start, end = item.replace(' ', '').split(',')
                    if len(start) > 10 or len(end) > 10:
                        contains_precise.append('precise')
                    else:
                        contains_precise.append('normal')
                elif len(item) > 10:
                    contains_precise.append('precise')
                else:
                    contains_precise.append('normal')
            # Handle a mix of normal and precise dates
            if 'normal' in contains_precise and 'precise' in contains_precise:
                d['normal'] = []
                d['precise'] = []
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        if len(start) > 10 or len(end) > 10:
                            d['precise'].append({'start': start, 'end': end})
                        else:
                            d['normal'].append({'start': start, 'end': end})
                    else:
                        if len(item) > 10:
                            d['precise'].append(item)
                        else:
                            d['normal'].append(item)
                new_date.append(d)
            # Handle only precise dates
            elif 'precise' in contains_precise:
                d['precise'] = []
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        d['precise'].append({'start': start, 'end': end})
                    else:
                        d['precise'].append(item)
                new_date.append(d)
            # Handle only normal dates
            else:
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        new_date.append({'start': start, 'end': end})
                    else:
                        new_date.append(item)

            return new_date

        def handle_submit(values):
            # Save the form values in a dict
            self.values = {'action': ConfigForm.action.value}
            self.values['title'] = ConfigForm.title.value 
            self.values['altTitle'] = ConfigForm.altTitle.value
            self.values['date'] = process_dates(ConfigForm.date.value) 
            self.values['description'] = ConfigForm.description.value 
            self.values['label'] = ConfigForm.label.value 
            self.values['notes'] = split_list(ConfigForm.notes.value) 
            if ConfigForm.action.value == 'Insert':
                self.values['datatype'] = ConfigForm.datatype.value 
                self.values['collection'] = ConfigForm.collection.value.strip() 
                self.values['relationships'] = split_list(ConfigForm.relationships.value) 
                self.values['rights'] = ConfigForm.rights.value.strip()
                self.values['OCR'] = ConfigForm.OCR.value
                self.values['source_files'] = split_list(ConfigForm.source_files.value)
                self.values['processes'] = split_list(ConfigForm.processes.value)
            if ConfigForm.action.value == 'Add to Existing Collection':
                self.values['datatype'] = ConfigForm.datatype.value 
                self.values['collection'] = ConfigForm.collection.value.strip()
                self.values['source_files'] = split_list(ConfigForm.source_files.value)
                self.values['processes'] = split_list(ConfigForm.processes.value)
            print('Configuration saved. Values will be available is the cells below.')
            print(self.values)
    
        # Initialise widgets in the container Box
        self.form = Box([self.action, self.datatype, self.collection, self.relationships, 
                         self.rights, self.ocr, self.source_files, self.processes, 
                         self.title, self.altTitle, self.date, self.description, self.label, 
                         self.notes],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form
        for item in box.children[2:6]:
            item.remove_class('visible').add_class('hidden')
        action_field = box.children[0].children[1]
        nodetype_field = box.children[1].children[1]

        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.button)
        action_field.observe(change_visibility)
        nodetype_field.observe(change_visibility, names='value')
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)